In [ ]:
from geo_dist_prep.data.populate_tree import populate_tree

globe = populate_tree('..')

In [ ]:
from geo_dist_prep.tree.globe import Globe


southport = 26700924
lat1, lon1 = 53.647599, -3.004175  # southport

newglobe = Globe()
address = list(newglobe.get_address(lat1, lon1))

# print(address)
# current = newglobe
# for pos in address:
#     current = current.add_child(pos)

southport = globe.find(address)

In [ ]:
region = southport.parent.parent.parent
region.parent.plot(depth=2)
region.plot()
#region.neighbours[1].plot(colour='red')
#print('siblings:', region.parent.children)
#print('neighbours:', region.neighbours)
#print('parent neighbours:', region.parent.neighbours)
#southport.parent.parent.neighbours[1].plot(colour='red')
#southport.parent.width * 113.5

address = (int(i) for i in ('0/3/3/2/0/0/2/1' '/-3/-3/-1/0/2').split('/'))
address = (int(i) for i in ('0/3/3/2/0/3/0/2/-0').split('/')) 
#right_region = globe.find(address)
#right_region.plot(depth=3, colour='red')

print("region:", repr(region))
#print("right region neightbours:", right_region.neighbours)
#

In [ ]:
southport.parent.parent.parent.parent.plot()